In [1]:
import os
base_directory = '/workspaces/streaming-prj'
os.chdir(base_directory)

send message

In [ ]:
import socket

# Setup UDP socket
UDP_IP = "127.0.0.1"  # Localhost (for communication between notebooks on the same machine)
UDP_PORT = 5005  # Port to use for communication

sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)  # Create UDP socket

# Message to send
message = "Hello from Notebook 1"

# Send the message
sock.sendto(message.encode(), (UDP_IP, UDP_PORT))
print(f"Sent message: {message}")
sock.close()


send larger content

In [ ]:
import socket
import json
import math

# Setup UDP socket
UDP_IP = "127.0.0.1"  # Localhost (for communication between notebooks on the same machine)
UDP_PORT = 5005  # Port to use for communication

sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)  # Create UDP socket

# Original message
message = "This is a much larger message. " * 50  # Simulate a larger message

# Convert the message to a JSON format
data = {"message": message}
json_data = json.dumps(data)

# Split the JSON data into smaller chunks
chunk_size = 512  # Define chunk size for transmission
chunks = [json_data[i:i + chunk_size] for i in range(0, len(json_data), chunk_size)]

# Send each chunk
for idx, chunk in enumerate(chunks):
    sock.sendto(chunk.encode(), (UDP_IP, UDP_PORT))
    print(f"Sent chunk {idx+1}/{len(chunks)}: {chunk[:50]}...")  # Show first 50 characters
sock.close()


send an image

In [ ]:
import socket
import os

# Setup UDP socket
UDP_IP = "127.0.0.1"  # Localhost (for communication between notebooks on the same machine)
UDP_PORT = 5005  # Port to use for communication

sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)  # Create UDP socket

# Path to the binary file (e.g., an image or audio file)
file_path = "image/example_image.jpg"  # Change this to any binary file

# Open the file in binary mode
with open(file_path, "rb") as file:
    # Read the file in chunks
    chunk_size = 1024  # 1 KB chunks
    chunk_number = 0
    while chunk := file.read(chunk_size):
        sock.sendto(chunk, (UDP_IP, UDP_PORT))
        chunk_number += 1
        print(f"Sent chunk {chunk_number} of size {len(chunk)} bytes")

# Send an 'END' signal to indicate the end of the transmission
sock.sendto(b'END', (UDP_IP, UDP_PORT))
print("Sent termination signal: 'END'")

sock.close()


send audio all at once

In [ ]:
import pygame
from pydub import AudioSegment
import io

# Initialize pygame mixer for audio playback
pygame.mixer.init()

# Load MP3 file using pydub
mp3_file = "/mnt/d/GitHub/Jenny.mp3"  # Replace with your file path

# Use pydub to read the MP3 file
audio = AudioSegment.from_mp3(mp3_file)

# Export audio to a format pygame can use (WAV in this case)
audio_wav = io.BytesIO()  # Use in-memory file
audio.export(audio_wav, format="wav")
audio_wav.seek(0)  # Rewind to the start of the file

# Load the in-memory WAV data into pygame mixer
pygame.mixer.music.load(audio_wav)

# Play the audio
pygame.mixer.music.play()

# Wait for the audio to finish
while pygame.mixer.music.get_busy():
    pygame.time.Clock().tick(10)


stream audio

In [ ]:
import socket
import io
import pygame
from pydub import AudioSegment

# Setup UDP socket
UDP_IP = "127.0.0.1"  # Localhost (for communication between notebooks on the same machine)
UDP_PORT = 5005  # Port to use for communication

sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)  # Create UDP socket

# Initialize pygame mixer for audio playback
pygame.mixer.init()

# Load MP3 file using pydub
mp3_file = "/mnt/d/GitHub/Jenny.mp3"  # Replace with your file path

# Use pydub to read the MP3 file
audio = AudioSegment.from_mp3(mp3_file)

# Convert audio to WAV format for easy chunking
audio_wav = io.BytesIO()
audio.export(audio_wav, format="wav")
audio_wav.seek(0)  # Rewind to the start of the file

# Send the audio data in chunks over UDP
chunk_size = 1024  # Adjust the chunk size as needed
chunk_number = 0

while chunk := audio_wav.read(chunk_size):
    sock.sendto(chunk, (UDP_IP, UDP_PORT))  # Send chunk via UDP
    chunk_number += 1
    print(f"Sent chunk {chunk_number} of size {len(chunk)} bytes")

# Send an 'END' signal to indicate the end of the transmission
sock.sendto(b'END', (UDP_IP, UDP_PORT))
print("Sent termination signal: 'END'")

sock.close()


stream audio with delay

In [ ]:
import socket
import io
import pygame
from pydub import AudioSegment
import time
import random

# Setup UDP socket
UDP_IP = "127.0.0.1"  # Localhost (for communication between notebooks on the same machine)
UDP_PORT = 5005  # Port to use for communication

sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)  # Create UDP socket

# Initialize pygame mixer for audio playback
pygame.mixer.init()

# Load MP3 file using pydub
mp3_file = "/mnt/d/GitHub/Jenny.mp3"  # Replace with your file path

# Use pydub to read the MP3 file
audio = AudioSegment.from_mp3(mp3_file)

# Convert audio to WAV format for easy chunking
audio_wav = io.BytesIO()
audio.export(audio_wav, format="wav")
audio_wav.seek(0)  # Rewind to the start of the file

# Send the audio data in chunks over UDP
chunk_size = 1024  # Adjust the chunk size as needed
chunk_number = 0
total_chunks = len(audio_wav.getvalue()) // chunk_size

# Simulate real-time by introducing a small delay between chunks
for _ in range(total_chunks):
    # Randomly decide to skip sending a chunk (simulate packet loss)
    if random.random() < 0.05:  # 5% chance of dropping a chunk
        print(f"Skipping chunk {chunk_number}")
        continue  # Skip this chunk
    
    chunk = audio_wav.read(chunk_size)
    sock.sendto(chunk, (UDP_IP, UDP_PORT))  # Send chunk via UDP
    chunk_number += 1
    print(f"Sent chunk {chunk_number} of size {len(chunk)} bytes")
    
    # Add a small delay to simulate real-time transmission
    time.sleep(0.05)  # Adjust this value based on your desired transmission speed

# Send an 'END' signal to indicate the end of the transmission
sock.sendto(b'END', (UDP_IP, UDP_PORT))
print("Sent termination signal: 'END'")

sock.close()


In [ ]:
import socket
import io
import pygame
from pydub import AudioSegment
import time
import random

# Setup UDP socket
UDP_IP = "127.0.0.1"  # Localhost (for communication between notebooks on the same machine)
UDP_PORT = 5005  # Port to use for communication

sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)  # Create UDP socket

# Initialize pygame mixer for audio playback
pygame.mixer.init()

# Load MP3 file using pydub
mp3_file = "/mnt/d/GitHub/Jenny.mp3"  # Replace with your file path

# Use pydub to read the MP3 file
audio = AudioSegment.from_mp3(mp3_file)

# Convert audio to WAV format for easy chunking (keep it as MP3 if you want)
audio_wav = io.BytesIO()
audio.export(audio_wav, format="wav")
audio_wav.seek(0)  # Rewind to the start of the file

# Send format information first (WAV in this case)
format_info = "WAV"  # or "MP3" depending on your format
sock.sendto(format_info.encode(), (UDP_IP, UDP_PORT))  # Send format info first
print(f"Sent format info: {format_info}")

# Send the audio data in chunks over UDP
chunk_size = 1024  # Adjust the chunk size as needed
chunk_number = 0
total_chunks = len(audio_wav.getvalue()) // chunk_size

# Simulate real-time by introducing a small delay between chunks
for _ in range(total_chunks):
    # Randomly decide to skip sending a chunk (simulate packet loss)
    if random.random() < 0.05:  # 5% chance of dropping a chunk
        print(f"Skipping chunk {chunk_number}")
        continue  # Skip this chunk
    
    chunk = audio_wav.read(chunk_size)
    sock.sendto(chunk, (UDP_IP, UDP_PORT))  # Send chunk via UDP
    chunk_number += 1
    print(f"Sent chunk {chunk_number} of size {len(chunk)} bytes")
    
    # Add a small delay to simulate real-time transmission
    time.sleep(0.05)  # Adjust this value based on your desired transmission speed

# Send an 'END' signal to indicate the end of the transmission
sock.sendto(b'END', (UDP_IP, UDP_PORT))
print("Sent termination signal: 'END'")

sock.close()


In [ ]:
import socket
import time
import pygame
from pydub import AudioSegment
import io

# Setup UDP socket
UDP_IP = "127.0.0.1"  # Localhost
UDP_PORT = 5006  # Port to use for communication

sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)  # Create UDP socket

# Initialize pygame mixer for audio playback
pygame.mixer.init()

# Load the audio file
audio_file = "/mnt/d/GitHub/Jenny.mp3"  # Replace with your MP3 or WAV file

# Use pydub to read the MP3 file (convert it to WAV)
audio = AudioSegment.from_mp3(audio_file)

# Ensure the audio is stereo (2 channels)
audio = audio.set_channels(2)  # Convert to stereo if it's mono

# Convert the audio to WAV format first
wav_data = io.BytesIO()
audio.export(wav_data, format="wav")  # Export as WAV
wav_data.seek(0)  # Rewind to start of data

# Read the WAV data as raw PCM
import wave

# Open the WAV data using the wave module to extract PCM
with wave.open(wav_data, 'rb') as wave_file:
    # Check that the file is stereo (2 channels) and 16-bit
    assert wave_file.getsampwidth() == 2  # 16-bit audio
    assert wave_file.getnchannels() == 2  # Stereo
    assert wave_file.getframerate() == 44100  # 44.1 kHz sample rate

    # Read the audio frames (raw PCM data)
    pcm_data = wave_file.readframes(wave_file.getnframes())

# Send the raw PCM data in chunks over UDP
chunk_size = 1024  # Adjust the chunk size as needed
for i in range(0, len(pcm_data), chunk_size):
    chunk = pcm_data[i:i + chunk_size]
    sock.sendto(chunk, (UDP_IP, UDP_PORT))  # Send chunk via UDP
    print(f"Sent chunk of size {len(chunk)} bytes")

    # Add a small delay to simulate real-time transmission
    time.sleep(0.05)  # 50ms delay, adjust for smoother real-time experience

# Send an 'END' signal to indicate the end of the transmission
sock.sendto(b'END', (UDP_IP, UDP_PORT))
print("Sent termination signal: 'END'")

sock.close()


new method

In [ ]:
import pygame
from pydub import AudioSegment
import io

# Initialize pygame mixer for audio playback
pygame.mixer.init(frequency=44100, size=-16, channels=2)  # 44.1 kHz, 16-bit stereo PCM
print("Pygame mixer initialized.")

# Load the audio file
audio_file = "/mnt/d/GitHub/Jenny.mp3"  # Replace with your MP3 file path

# Use pydub to read the MP3 file
audio = AudioSegment.from_mp3(audio_file)

# Ensure the audio is stereo (2 channels)
audio = audio.set_channels(2)  # Convert to stereo if it's mono

# Convert the audio to WAV format first
wav_data = io.BytesIO()
audio.export(wav_data, format="wav")  # Export as WAV
wav_data.seek(0)  # Rewind to the start of data

# Now we read the WAV data and load it into pygame.mixer.music
try:
    pygame.mixer.music.load(wav_data)  # Load the WAV data into pygame mixer
    print("WAV file loaded into pygame.")
except pygame.error as e:
    print(f"Error loading WAV file: {e}")

# Play the audio
pygame.mixer.music.play()
print("Audio playing...")

# Wait for the audio to finish playing
while pygame.mixer.music.get_busy():
    pygame.time.Clock().tick(10)

print("Audio playback finished.")


single chunk playback test

In [ ]:
import pygame
from pydub import AudioSegment
import io

# Initialize pygame mixer for audio playback
pygame.mixer.init(frequency=44100, size=-16, channels=2)  # 44.1 kHz, 16-bit stereo PCM
print("Pygame mixer initialized.")

# Load the audio file
audio_file = "/mnt/d/GitHub/Jenny.mp3"  # Replace with your MP3 file path

# Use pydub to read the MP3 file
audio = AudioSegment.from_mp3(audio_file)

# Ensure the audio is stereo (2 channels)
audio = audio.set_channels(2)  # Convert to stereo if it's mono

# Convert the audio to WAV format first
wav_data = io.BytesIO()
audio.export(wav_data, format="wav")  # Export as WAV
wav_data.seek(0)  # Rewind to the start of data

# Break the audio into chunks (1-second chunks for simplicity)
chunk_length_ms = 1000  # 1-second chunks
chunks = []
for start_ms in range(0, len(audio), chunk_length_ms):
    chunk = audio[start_ms:start_ms + chunk_length_ms]
    chunks.append(chunk)

# Let's play the first chunk to verify
first_chunk = chunks[4]

# Convert the first chunk to WAV format
chunk_data = io.BytesIO()
first_chunk.export(chunk_data, format="wav")
chunk_data.seek(0)  # Rewind to the start of the chunk data

# Now, load and play the chunk using pygame's mixer
try:
    pygame.mixer.music.load(chunk_data)  # Load the chunk into pygame mixer
    pygame.mixer.music.play()  # Play the audio chunk
    print("Playing the first chunk...")

    # Wait for the audio to finish playing
    while pygame.mixer.music.get_busy():
        pygame.time.Clock().tick(10)

    print("First chunk playback finished.")
except pygame.error as e:
    print(f"Error loading audio: {e}")


send single chunk test

In [ ]:
import socket
from pydub import AudioSegment
import io

# Setup UDP socket
UDP_IP = "127.0.0.1"  # Localhost (for communication between notebooks on the same machine)
UDP_PORT = 5006  # Port to use for communication

sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)  # Create UDP socket

# Load the audio file
audio_file = "/mnt/d/GitHub/Jenny.mp3"  # Replace with your MP3 file path

# Use pydub to read the MP3 file
audio = AudioSegment.from_mp3(audio_file)

# Ensure the audio is stereo (2 channels)
audio = audio.set_channels(2)  # Convert to stereo if it's mono

# Convert the audio to WAV format first
wav_data = io.BytesIO()
audio.export(wav_data, format="wav")  # Export as WAV
wav_data.seek(0)  # Rewind to the start of data

# Break the audio into chunks (1-second chunks for simplicity)
chunk_length_ms = 250  # 1-second chunks
chunks = []
for start_ms in range(0, len(audio), chunk_length_ms):
    chunk = audio[start_ms:start_ms + chunk_length_ms]
    chunks.append(chunk)

# Now let's send the first chunk
first_chunk = chunks[1]

# Convert the first chunk to WAV format
chunk_data = io.BytesIO()
first_chunk.export(chunk_data, format="wav")
chunk_data.seek(0)  # Rewind to the start of the chunk data

# Check the size of the chunk before sending
chunk_size = len(chunk_data.getvalue())  # Size in bytes
print(f"Chunk size: {chunk_size} bytes")

# Adjust the chunk size if needed (make sure it's small enough for UDP)
max_udp_packet_size = 65507  # Maximum UDP packet size (including headers)
if chunk_size > max_udp_packet_size:
    print(f"Chunk size is too large. Reducing size...")
    # Reduce the chunk size by sending only a portion of it
    chunk_data = io.BytesIO(chunk_data.read(max_udp_packet_size))

# Send the first chunk over UDP
sock.sendto(chunk_data.read(), (UDP_IP, UDP_PORT))  # Send chunk via UDP
print(f"Sent the first chunk of size {len(chunk_data.getvalue())} bytes")

# Send an 'END' signal to indicate the end of the transmission
# sock.sendto(b'END', (UDP_IP, UDP_PORT))
# print("Sent termination signal: 'END'")

sock.close()


In [10]:
# Final version

import socket
from pydub import AudioSegment
import io
import time

# Setup UDP socket
UDP_IP = "127.0.0.1"  # Localhost (for communication between notebooks on the same machine)
UDP_PORT = 5006  # Port to use for communication

sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)  # Create UDP socket

# Load the audio file
audio_file = "audio/Jenny.mp3"  # Replace with your MP3 file path

# Use pydub to read the MP3 file
audio = AudioSegment.from_mp3(audio_file)

# Ensure the audio is stereo (2 channels)
audio = audio.set_channels(2)  # Convert to stereo if it's mono

# Convert the audio to WAV format first
wav_data = io.BytesIO()
audio.export(wav_data, format="wav")  # Export as WAV
wav_data.seek(0)  # Rewind to the start of data

# Break the audio into chunks (e.g., 250 ms chunks)
chunk_length_ms = 250  # 250 ms per chunk
chunks = []
for start_ms in range(0, len(audio), chunk_length_ms):
    chunk = audio[start_ms:start_ms + chunk_length_ms]
    chunks.append(chunk)

# Send each chunk with a delay equal to the chunk's length in ms
for chunk in chunks:
    # Convert the chunk to WAV format
    chunk_data = io.BytesIO()
    chunk.export(chunk_data, format="wav")
    chunk_data.seek(0)  # Rewind to the start of the chunk data

    # Check the size of the chunk before sending
    chunk_size = len(chunk_data.getvalue())  # Size in bytes
    print(f"Chunk size: {chunk_size} bytes")

    # Send the chunk over UDP
    sock.sendto(chunk_data.read(), (UDP_IP, UDP_PORT))  # Send chunk via UDP
    print(f"Sent chunk of size {len(chunk_data.getvalue())} bytes")

    # Calculate the delay (in seconds) based on the chunk length in milliseconds
    delay_seconds = chunk_length_ms / 1000.0
    print(f"Waiting for {delay_seconds} seconds before sending the next chunk...")
    time.sleep(delay_seconds)  # Delay between sending chunks

# Close the socket
sock.close()


Chunk size: 44144 bytes
Sent chunk of size 44144 bytes
Waiting for 0.25 seconds before sending the next chunk...
Chunk size: 44144 bytes
Sent chunk of size 44144 bytes
Waiting for 0.25 seconds before sending the next chunk...
Chunk size: 44144 bytes
Sent chunk of size 44144 bytes
Waiting for 0.25 seconds before sending the next chunk...
Chunk size: 44144 bytes
Sent chunk of size 44144 bytes
Waiting for 0.25 seconds before sending the next chunk...
Chunk size: 44144 bytes
Sent chunk of size 44144 bytes
Waiting for 0.25 seconds before sending the next chunk...
Chunk size: 44144 bytes
Sent chunk of size 44144 bytes
Waiting for 0.25 seconds before sending the next chunk...
Chunk size: 44144 bytes
Sent chunk of size 44144 bytes
Waiting for 0.25 seconds before sending the next chunk...
Chunk size: 44144 bytes
Sent chunk of size 44144 bytes
Waiting for 0.25 seconds before sending the next chunk...
Chunk size: 44144 bytes
Sent chunk of size 44144 bytes
Waiting for 0.25 seconds before sending t

In [8]:
import socket
from pydub import AudioSegment
import io
import time

# Setup UDP socket
UDP_IP = "127.0.0.1"  # Localhost (for communication between notebooks on the same machine)
UDP_PORT = 5006  # Port to use for communication

sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)  # Create UDP socket

# Load the audio file
audio_file = "audio/Jenny.mp3"  # Replace with your MP3 file path

# Use pydub to read the MP3 file
audio = AudioSegment.from_mp3(audio_file)

# Ensure the audio is stereo (2 channels)
audio = audio.set_channels(2)  # Convert to stereo if it's mono

# Convert the audio to WAV format first
wav_data = io.BytesIO()
audio.export(wav_data, format="wav")  # Export as WAV
wav_data.seek(0)  # Rewind to the start of data

# Break the audio into chunks (e.g., 250 ms chunks)
chunk_length_ms = 250  # 250 ms per chunk
chunks = []
for start_ms in range(0, len(audio), chunk_length_ms):
    chunk = audio[start_ms:start_ms + chunk_length_ms]
    chunks.append(chunk)

# Send each chunk with a delay equal to the chunk's length in ms, accounting for transmission time
for chunk in chunks:
    # Convert the chunk to WAV format
    chunk_data = io.BytesIO()
    chunk.export(chunk_data, format="wav")
    chunk_data.seek(0)  # Rewind to the start of the chunk data

    # Check the size of the chunk before sending
    chunk_size = len(chunk_data.getvalue())  # Size in bytes
    print(f"Chunk size: {chunk_size} bytes")

    # Send the chunk over UDP
    start_time = time.time()  # Start time before sending
    sock.sendto(chunk_data.read(), (UDP_IP, UDP_PORT))  # Send chunk via UDP
    print(f"Sent chunk of size {len(chunk_data.getvalue())} bytes")

    # Calculate the actual time taken to send the chunk
    send_duration = time.time() - start_time  # Time taken to send the chunk
    print(f"Time taken to send chunk: {send_duration:.4f} seconds")

    # Calculate the delay to maintain chunk length timing (in seconds)
    delay_seconds = (chunk_length_ms / 1000.0) - send_duration
    if delay_seconds > 0:
        print(f"Waiting for {delay_seconds:.4f} seconds before sending the next chunk...")
        time.sleep(delay_seconds)  # Delay between sending chunks to match chunk duration

# Close the socket
sock.close()


Chunk size: 44144 bytes
Sent chunk of size 44144 bytes
Time taken to send chunk: 0.0005 seconds
Waiting for 0.2495 seconds before sending the next chunk...
Chunk size: 44144 bytes
Sent chunk of size 44144 bytes
Time taken to send chunk: 0.0003 seconds
Waiting for 0.2497 seconds before sending the next chunk...
Chunk size: 44144 bytes
Sent chunk of size 44144 bytes
Time taken to send chunk: 0.0003 seconds
Waiting for 0.2497 seconds before sending the next chunk...
Chunk size: 44144 bytes
Sent chunk of size 44144 bytes
Time taken to send chunk: 0.0004 seconds
Waiting for 0.2496 seconds before sending the next chunk...
Chunk size: 44144 bytes
Sent chunk of size 44144 bytes
Time taken to send chunk: 0.0003 seconds
Waiting for 0.2497 seconds before sending the next chunk...
Chunk size: 44144 bytes
Sent chunk of size 44144 bytes
Time taken to send chunk: 0.0005 seconds
Waiting for 0.2495 seconds before sending the next chunk...
Chunk size: 44144 bytes
Sent chunk of size 44144 bytes
Time take